# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [2]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import pdist, squareform
from pprint import pprint


In [3]:
data = pd.read_csv('../data/customer_product_sales.csv')
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
quant= data.groupby(['CustomerID','ProductName']).agg({'Quantity':'sum'})
quant

Quantity
CustomerID ProductName                                
33         Apricots - Dried                          1
           Assorted Desserts                         1
           Bandage - Flexible Neon                   1
           Bar Mix - Pina Colada, 355 Ml             1
           Beans - Kidney, Canned                    1
           Beef - Chuck, Boneless                    1
           Beef - Prime Rib Aaa                      1
           Beer - Original Organic Lager             1
           Beer - Rickards Red                       1
           Black Currants                            1
           Blackberries                              1
           Bread - Italian Roll With Herbs           1
           Bread Foccacia Whole                      1
           Cake - Box Window 10x10x2.5               1
           Cassis                                    1
           Cheese - Cottage Cheese                   1
           Crab - Imitation Flakes                   1
           Fondant - Icing                           1
           French Pastry - Mini Chocolate            1
           General Purpose Trigger                   1
           Grapes - Red                              1
           Grouper - Fresh                           2
           Hersey Shakes                             1
           Ice Cream Bar - Drumstick                 1
           Lamb - Pieces, Diced                      1
           Lettuce - Spring Mix                      2
           Mangoes                                   1
           Muffin Batt - Choc Chk                    1
           Onions - Vidalia                          1
           Pasta - Cheese / Spinach Bauletti         1
...                                                ...
98200      Onion Powder                             25
           Orange - Canned, Mandarin                25
           Pasta - Detalini, White, Fresh           25
           Puree - Passion Fruit                    25
           Raspberries - Fresh                      25
           Rice - Long Grain                        25
           Rosemary - Dry                           25
           Rosemary - Primerba, Paste               50
           Rum - Coconut, Malibu                    25
           Sage - Ground                            25
           Sausage - Liver                          25
           Sea Bass - Whole                         50
           Seedlings - Mix, Organic                 25
           Sole - Dover, Whole, Fresh               25
           Soup - Campbells, Lentil                 50
           Soup Campbells - Italian Wedding         25
           Soup Knorr Chili With Beans              25
           Soupfoamcont12oz 112con                  25
           Spinach - Baby                           25
           Table Cloth 54x72 White                  25
           Tea - English Breakfast                  25
           Tilapia - Fillets                        25
           Tomatoes Tear Drop                       25
           Truffle Cups - Brown                     25
           Vaccum Bag 10x13                         25
           Vol Au Vents                             50
           Wasabi Powder                            25
           Wine - Fume Blanc Fetzer                 25
           Wine - Hardys Bankside Shiraz            25
           Yogurt - French Vanilla                  25

[63628 rows x 1 columns]

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [5]:
table = pd.pivot_table(data, values='Quantity', index=['ProductName'], columns=['CustomerID'], aggfunc='sum')
table = table.fillna(0)
table

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Apricots - Halves,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,50.0,25.0,0.0,25.0,25.0
Apricots Fresh,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0
Arizona - Green Tea,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0
Artichokes - Jerusalem,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [6]:
sq= squareform(pdist(table.T,'euclidean'))
sq.shape, table.shape

((1000, 1000), (452, 1000))

In [7]:
distances = pd.DataFrame(1/(1 + sq), index=table.columns, columns=table.columns)
distances

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349
464,0.082709,0.075255,0.080350,0.078187,0.078711,1.000000,0.074798,0.084626,0.080070,0.078187,...,0.004816,0.004687,0.004416,0.005038,0.004308,0.004533,0.004578,0.004363,0.004167,0.004347
477,0.074573,0.075956,0.076923,0.075025,0.075025,0.074798,1.000000,0.078977,0.075720,0.074573,...,0.004823,0.004688,0.004426,0.005037,0.004321,0.004534,0.004589,0.004366,0.004168,0.004338
639,0.083020,0.076435,0.080634,0.082403,0.082403,0.084626,0.078977,1.000000,0.085983,0.083020,...,0.004816,0.004682,0.004412,0.005035,0.004308,0.004526,0.004581,0.004363,0.004176,0.004335
649,0.081503,0.077674,0.082100,0.077171,0.078187,0.080070,0.075720,0.085983,1.000000,0.079792,...,0.004810,0.004676,0.004414,0.005035,0.004314,0.004528,0.004585,0.004369,0.004172,0.004329


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [8]:
simil=distances.iloc[0].sort_values(ascending=False)[1:6]
similarities = list(distances.iloc[0].sort_values(ascending=False)[1:6])
sim= list(simil.index)
sim

[264, 3535, 3317, 2503, 3305]

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [9]:
final= quant.loc[sim]
final

Quantity
CustomerID ProductName                              
264        Apricots - Halves                       1
           Apricots Fresh                          1
           Bacardi Breezer - Tropical              1
           Bagel - Plain                           1
           Banana - Leaves                         1
           Bananas                                 1
           Bandage - Flexible Neon                 1
           Beef - Tenderlion, Center Cut           1
           Berry Brulee                            1
           Bread - Roll, Canadian Dinner           1
           Bread Foccacia Whole                    1
           Cheese - Brie, Triple Creme             1
           Cheese - Parmesan Cubes                 1
           Cheese Cloth No 100                     1
           Chocolate - Dark                        2
           Cocoa Butter                            1
           Coffee - Dark Roast                     1
           Coffee Decaf Colombian                  1
           Cookie Chocolate Chip With              1
           Crab - Dungeness, Whole                 1
           Crackers - Trio                         1
           Creme De Banane - Marie                 1
           Cumin - Whole                           1
           Duck - Breast                           1
           Flour - Teff                            1
           Grouper - Fresh                         1
           Guinea Fowl                             2
           Hinge W Undercut                        1
           Langers - Ruby Red Grapfruit            1
           Lemonade - Natural, 591 Ml              1
...                                              ...
3535       Juice - Orange                          1
           Ketchup - Tomato                        1
           Lamb - Ground                           1
           Lamb - Pieces, Diced                    1
           Milk - 2%                               1
           Milk Powder                             1
           Muffin Batt - Blueberry Passion         1
           Ocean Spray - Kiwi Strawberry           1
           Ocean Spray - Ruby Red                  1
           Pasta - Orecchiette                     1
           Pepper - Black, Whole                   1
           Sausage - Breakfast                     1
           Scallops - 10/20                        1
           Scallops 60/80 Iqf                      1
           Sea Bass - Whole                        1
           Sherry - Dry                            1
           Sole - Dover, Whole, Fresh              1
           Soup - Campbells Bean Medley            1
           Soupfoamcont12oz 112con                 1
           Steam Pan - Half Size Deep              1
           Sugar - Fine                            1
           Table Cloth 54x72 White                 1
           Tahini Paste                            1
           Tofu - Firm                             1
           Towels - Paper / Kraft                  1
           Tray - 16in Rnd Blk                     1
           Wanton Wrap                             1
           Wine - Blue Nun Qualitatswein           1
           Wine - Magnotta, Merlot Sr Vqa          1
           Yoghurt Tubes                           1

[270 rows x 1 columns]

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [10]:
step6= final.groupby(['ProductName']).agg({'Quantity':'sum'}).sort_values(['Quantity'],ascending = False)
step6

,Quantity
ProductName,
Butter - Unsalted,3
Wine - Ej Gallo Sierra Valley,3
Towels - Paper / Kraft,3
Soup - Campbells Bean Medley,3
Wine - Blue Nun Qualitatswein,3
"Pepper - Black, Whole",3
Chicken - Soup Base,2
Cod - Black Whole Fillet,2
Chocolate - Dark,2


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

#### Merge the ranked products data frame with the customer product matrix on the ProductName field.

In [11]:
step7 = step6.merge(table[33], on='ProductName')

#### Filter for records where the chosen customer has not purchased the product.


In [12]:
st7 = step7.loc[step7[33]==0.0]

#### Show the top 5 results.

In [13]:
st7.head()

,Quantity,33
ProductName,,
Butter - Unsalted,3,0.0
Wine - Ej Gallo Sierra Valley,3,0.0
Soup - Campbells Bean Medley,3,0.0
Wine - Blue Nun Qualitatswein,3,0.0
Chicken - Soup Base,2,0.0


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

#### Create an empty dictionary that will hold the recommendations for all customers.

#### Create a list of unique CustomerIDs to iterate over

#### Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created

In [14]:
recommend = {}
customers = set(list(data['CustomerID']))
for cu in customers:
    simila = distances[cu].sort_values(ascending=False)[1:6]
    simi = list(simila.index)
    finali= quant.loc[simi]
    #recom = finali.groupby(['ProductName'], as_index=False).agg({'Quantity':'sum'}).sort_values(['Quantity'],ascending = False)
    recom = finali.reset_index().groupby(['ProductName']).agg({'Quantity':'sum'}).sort_values(['Quantity'],ascending = False)
    recomm = recom.merge(table[cu], on='ProductName')
    filtered = recomm.loc[recomm[cu]==0.0].sort_values(by='Quantity', ascending= False)[1:6]
    pasowhat =list(filtered.index)
    recommend[cu]=pasowhat

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [15]:
df=pd.DataFrame(recommend)
df= df.T
df

,0,1,2,3,4
83973,Veal - Slab Bacon,Bandage - Fexible 1x3,Ezy Change Mophandle,Chicken - Wieners,Puree - Passion Fruit
59399,Towels - Paper / Kraft,Knife Plastic - White,Nantucket - Pomegranate Pear,Snapple - Iced Tea Peach,Cookie Chocolate Chip With
92168,Snapple Lemon Tea,"Veal - Brisket, Provimi,bnls",Spinach - Baby,Pop Shoppe Cream Soda,Bread - Calabrese Baguette
49159,Cookies - Assorted,Puree - Mocha,Cheese - Parmesan Grated,Cheese - Victor Et Berthold,"Chicken - Leg, Boneless"
18441,Beef - Texas Style Burger,Anchovy Paste - 56 G Tube,Flour - Pastry,Foam Cup 6 Oz,"Garlic - Primerba, Paste"
22536,Cheese - Mozzarella,Cheese Cloth No 100,Bread - Hot Dog Buns,Table Cloth - 53x69 Colour,Banana - Leaves
86028,Vinegar - Tarragon,Pork - Hock And Feet Attached,"Orange - Canned, Mandarin","Pepper - Black, Whole",Beans - Wax
75791,Vaccum Bag 10x13,Turkey - Oven Roast Breast,Mussels - Cultivated,Bread - Calabrese Baguette,Grouper - Fresh
96272,Guinea Fowl,Bread Crumbs - Panko,Sunflower Seed Raw,Bread - English Muffin,Sprouts - Alfalfa
32785,Juice - V8 Splash,Wine - Redchard Merritt,Puree - Passion Fruit,Juice - Orange,Pate - Cognac


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [16]:
sq1= squareform(pdist(table.T,'cosine'))
distances1 = pd.DataFrame(1/(1 + sq1), index=table.columns, columns=table.columns)
distances1


CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.530003,0.563492,0.543546,0.542393,0.541054,0.522016,0.530783,0.533096,0.535293,...,0.530306,0.517987,0.527662,0.511471,0.534181,0.513809,0.528795,0.520208,0.529481,0.527142
200,0.530003,1.000000,0.537166,0.535098,0.521226,0.519193,0.556945,0.516729,0.536350,0.541451,...,0.545224,0.533067,0.555246,0.543554,0.530308,0.521925,0.573177,0.537284,0.526167,0.527280
264,0.563492,0.537166,1.000000,0.531963,0.550168,0.525368,0.538967,0.514932,0.537046,0.524193,...,0.546880,0.525550,0.535062,0.531806,0.553085,0.560225,0.548538,0.538340,0.554929,0.527142
356,0.543546,0.535098,0.531963,1.000000,0.525982,0.523986,0.536792,0.540890,0.516901,0.519482,...,0.536321,0.520565,0.533119,0.534066,0.528874,0.530406,0.520138,0.536202,0.544727,0.535974
412,0.542393,0.521226,0.550168,0.525982,1.000000,0.534073,0.542308,0.547743,0.530459,0.543050,...,0.527903,0.565369,0.528847,0.529278,0.541674,0.536686,0.519628,0.538710,0.546943,0.552636
464,0.541054,0.519193,0.525368,0.523986,0.534073,1.000000,0.528047,0.546574,0.528571,0.527240,...,0.537619,0.544446,0.534295,0.535278,0.526394,0.542873,0.520835,0.533856,0.528842,0.552221
477,0.522016,0.556945,0.538967,0.536792,0.542308,0.528047,1.000000,0.547544,0.534712,0.536302,...,0.543155,0.541855,0.545908,0.530540,0.544731,0.540381,0.534221,0.535601,0.527943,0.532238
639,0.530783,0.516729,0.514932,0.540890,0.547743,0.546574,0.547544,1.000000,0.555194,0.548386,...,0.539687,0.538500,0.528522,0.532803,0.527813,0.533190,0.525789,0.535704,0.548943,0.531706
649,0.533096,0.536350,0.537046,0.516901,0.530459,0.528571,0.534712,0.555194,1.000000,0.538236,...,0.529649,0.528778,0.530655,0.531114,0.537025,0.535224,0.531918,0.544926,0.539210,0.519657


In [19]:
recommend1 = {}
customers1 = set(list(data['CustomerID']))
for cu in customers1:
    simila1 = distances1[cu].sort_values(ascending=False)[1:6]
    simi1 = list(simila1.index)
    finali1= quant.loc[simi1]
    recom1 = finali1.reset_index().groupby(['ProductName']).agg({'Quantity':'sum'}).sort_values(['Quantity'],ascending = False)
    recomm1 = recom1.merge(table[cu], on='ProductName')
    filtered1 = recomm1.loc[recomm1[cu]==0.0].sort_values(by='Quantity', ascending= False)[1:6]
    pasowhat1 =list(filtered1.index)
    recommend1[cu]=pasowhat1

In [20]:
df1=pd.DataFrame(recommend1)
df1= df1.T
df1

,0,1,2,3,4
83973,Flour - Pastry,Puree - Passion Fruit,Lettuce - Spring Mix,Broom - Corn,"Cheese - Brie,danish"
59399,Anchovy Paste - 56 G Tube,Garlic - Peeled,Broom - Corn,Water - Aquafina Vitamin,Cake - Mini Cheesecake
92168,Tuna - Salad Premix,Sage - Ground,Apricots Fresh,Beef - Inside Round,Nantucket - Pomegranate Pear
49159,Beer - Blue,Peas - Frozen,Lamb - Ground,Beef - Ground Medium,"Beef - Chuck, Boneless"
18441,Whmis - Spray Bottle Trigger,Mussels - Cultivated,Ecolab - Lime - A - Way 4/4 L,Milk - 2%,Tea - Jasmin Green
22536,Muffin Batt - Choc Chk,Bread Crumbs - Panko,Spinach - Baby,Longos - Grilled Chicken With,Wine - White Cab Sauv.on
86028,Pail With Metal Handle 16l White,Sunflower Seed Raw,"Appetizer - Mini Egg Roll, Shrimp",Sausage - Breakfast,"Turkey - Whole, Fresh"
75791,"Placemat - Scallop, White",Beef - Ground Medium,Milk - 2%,Sausage - Liver,Beef - Top Sirloin - Aaa
96272,Fondant - Icing,Jolt Cola - Electric Blue,Lettuce - Spring Mix,Bread - Rye,Ocean Spray - Kiwi Strawberry
32785,Pail With Metal Handle 16l White,Thermometer Digital,Initation Crab Meat,Longos - Grilled Chicken With,Mayonnaise - Individual Pkg
